<a href="https://colab.research.google.com/github/myatsupaing-mint/GenAI/blob/main/GenAI_Day7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00


In [2]:
pip install --upgrade datasets fsspec

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_id = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
test_data = "I like TWS very much. But I like Seventeen more"
from transformers import pipeline
pipe = pipeline("text-classification", tokenizer = tokenizer, model = model)
output = pipe(test_data)
output

Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.5943374633789062}]

In [10]:
if output[0]['label'] == 'LABEL_0':
  print("The result is negative.")
else:
  print("The result is positive.")

The result is positive.


In [11]:
#Pytorch
tokenized_input = tokenizer(test_data, padding = True, truncation = True, return_tensors = "pt")
tokenized_input

{'input_ids': tensor([[ 101, 1045, 2066, 1056, 9333, 2200, 2172, 1012, 2021, 1045, 2066, 9171,
         2062,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [15]:
tokenized_input = {k: v.to(model.device) for k, v in tokenized_input.items()}

In [16]:
import torch
model.eval()
with torch.no_grad():
  output = model(**tokenized_input)

In [17]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1969,  0.1850]], device='cuda:0'), hidden_states=None, attentions=None)

In [18]:
output = output.logits
output = torch.softmax(output, dim = -1)
output

tensor([[0.4057, 0.5943]], device='cuda:0')

In [19]:
predicted_class = torch.argmax(output, dim = -1)
predicted_class

tensor([1], device='cuda:0')

In [20]:
class_label = ['neg', 'pos']
final_output = class_label[predicted_class]
final_output

'pos'

In [21]:
test = ["I’ve been exploring new ways to apply data science to real-world problems.",
        "Sometimes the best learning happens when things don’t go as planned.",
        "Even small steps forward are still progress."]

In [24]:
def token_func(data):
  return tokenizer(data, truncation = True, padding = True, return_tensors = 'pt')


tokenized_data = token_func(test)
tokenized_data

{'input_ids': tensor([[  101,  1045,  1521,  2310,  2042, 11131,  2047,  3971,  2000,  6611,
          2951,  2671,  2000,  2613,  1011,  2088,  3471,  1012,   102],
        [  101,  2823,  1996,  2190,  4083,  6433,  2043,  2477,  2123,  1521,
          1056,  2175,  2004,  3740,  1012,   102,     0,     0,     0],
        [  101,  2130,  2235,  4084,  2830,  2024,  2145,  5082,  1012,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [26]:
tokenized_data = {k: v.to(model.device) for k, v in tokenized_data.items()}

In [34]:
with torch.no_grad():
  result = model(**tokenized_data).logits
  result = torch.softmax(result, dim = -1)
  print(result)

tensor([[0.4195, 0.5805],
        [0.4335, 0.5665],
        [0.4015, 0.5985]], device='cuda:0')


In [35]:
prediction = torch.argmax(result, dim = -1)
prediction

tensor([1, 1, 1], device='cuda:0')

In [43]:
data_output = [class_label[pred.item()] for pred in prediction]
data_output

['pos', 'pos', 'pos']

Inferencing with Tensorflow

In [47]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained(model_id)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
text = "I hate doing the dishes."
tfTokenized = tokenizer(text, return_tensors = 'tf')
tfOutput = model(tfTokenized)
tfOutput

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.00776964, -0.01006192]], dtype=float32)>, hidden_states=None, attentions=None)

In [82]:
tfLogits = tfOutput.logits
tfLogits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.00776964, -0.01006192]], dtype=float32)>

In [84]:
tfProb = tf.nn.softmax(tfLogits)
tfProb

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.5044578 , 0.49554223]], dtype=float32)>

In [87]:
tfFinal = tf.argmax(tfProb).numpy()[0]
tfFinal

np.int64(0)

In [88]:
class_label[tfFinal]

'neg'

# Fine tuning

In [4]:
from datasets import load_dataset
rotten = load_dataset("rotten_tomatoes")
rotten

README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [6]:
def compute_length(data):
  return{'length' : len(data['text'])}

rotten['train'] = rotten['train'].map(compute_length)
rotten['train']

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'length'],
    num_rows: 8530
})

In [98]:
max(rotten['train']['length'])

267

In [23]:
def tokenized(data):
  return tokenizer(data['text'], truncation = True, padding = 'max_length', max_length = 267)

trained_token = rotten['train'].map(tokenized)
trained_token

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'length', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 8530
})

In [24]:
test_token = rotten['test'].map(tokenized)
vali_token = rotten['validation'].map(tokenized)

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

In [10]:
import evaluate
accuracy = evaluate.load("accuracy")

In [11]:
import numpy as np

def  compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis = 1)
  return accuracy.compute(predictions = predictions, references = labels)

In [12]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [17]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels = 2, id2label = id2label, label2id = label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from huggingface_hub import login
login()

In [15]:
pip install -U transformers

In [20]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [25]:
training_args = TrainingArguments(
    output_dir = "rotten_tomatoes",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 2,
    weight_decay = 0.01,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    report_to = "none",

)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = trained_token,
    eval_dataset = vali_token,
    processing_class = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics

)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.388100,0.356036,0.852720
2,0.211300,0.405467,0.862101


TrainOutput(global_step=1068, training_loss=0.2918925794322839, metrics={'train_runtime': 893.4055, 'train_samples_per_second': 19.095, 'train_steps_per_second': 1.195, 'total_flos': 2340773670675600.0, 'train_loss': 0.2918925794322839, 'epoch': 2.0})

In [26]:
model.push_to_hub("myatsu/bert-base-uncased-rotten-tomatoes-sentiment")
tokenizer.push_to_hub("myatsu/bert-base-uncased-rotten-tomatoes-sentiment")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/myatsu/bert-base-uncased-rotten-tomatoes-sentiment/commit/3bc73e19c2b9d787fabb72d56e8515de9804655c', commit_message='Upload tokenizer', commit_description='', oid='3bc73e19c2b9d787fabb72d56e8515de9804655c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/myatsu/bert-base-uncased-rotten-tomatoes-sentiment', endpoint='https://huggingface.co', repo_type='model', repo_id='myatsu/bert-base-uncased-rotten-tomatoes-sentiment'), pr_revision=None, pr_num=None)